In [4]:
import selenium
from selenium.webdriver.common.by import By
import pandas as pd
import re
from datetime import date, timedelta
import time
from selenium import webdriver


df_regions = pd.read_csv('../Data/regions_liens.csv', sep=',', encoding='utf-8')
region_url = df_regions['Lien'].unique()

# Prendre un region pour le test
for region in region_url[:1]:
    
    driver = webdriver.Chrome()
    url = region
    driver.get(url)
    button_refuse_cookie = driver.find_element(By.CLASS_NAME, 'VfPpkd-LgbsSe.VfPpkd-LgbsSe-OWXEXe-k8QpJ.VfPpkd-LgbsSe-OWXEXe-dgl2Hf.nCP5yc.AjY5Oe.DuMIQc.LQeN7.Nc7WLe').click()
    time.sleep(2)
    
    # Lire le fichier service et prendre les nom des services
    df_services = pd.read_csv('../Data/services.csv', sep=';', encoding='utf-8')
    services = df_services['Services publics'].unique()


    # Creation des listes vides
    comments_list_tout = []
    services_list_tout = []
    telephone_list_tout = []
    localisation_list_tout = []
    dates_list_tout = []
    auteur_list_tout = []
    etoile_list_tout = []
    region_tout = []
    
    
    # Rechercher les services dans toutes les regions de la France, 
    # pour notre exemple, on a prit POLE EMPLOI en île-de-France
    
    for service in services[:1]:
        search = driver.find_element(By.ID, 'searchboxinput')
        btn_search = driver.find_element(By.ID, 'searchbox-searchbutton')
        if search:
            search.clear()
        search.send_keys(service)
        time.sleep(2)
        btn_search.click()
        
        time.sleep(2)
        
        if driver.find_elements(By.CLASS_NAME, 'Nv2PK.tH5CWc.THOPZb'):
        # La barre d'affichage des services
            scrollable_div_list_services= driver.find_elements(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd')
            scrollable_div_service = driver.find_element(By.CSS_SELECTOR, '#QA0Szd > div > div > div.w6VYqd > div.bJzME.tTVLSc > div > div.e07Vkf.kA9KIf > div > div > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd')

            # if scrollable_div_list_services:
            old_freq = 0
            freq = 0

            # Scroller pour avoir la totaliter des services dans chaque region de la France
            while True:
                driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div_service)
                time.sleep(2)

                for elt in scrollable_div_list_services:
                    freq = elt.text.count('Itinéraires')

                if freq == old_freq:
                    break

                old_freq = freq
            
            # scrollable = driver.find_elements(By.CLASS_NAME, "Nv2PK.Q2HXcd.THOPZb")
            scrollable_tous_services = driver.find_elements(By.CLASS_NAME, "hfpxzc")
            time.sleep(2)
            
            for i in scrollable_tous_services:

                # clique sur les services
                # driver.execute_script("arguments[0].click();", i)
                element_click = i.click()
                time.sleep(2)
                
                # Coordonnées du service
                # coordonnee = driver.find_elements(By.CLASS_NAME,'Io6YTe.fontBodyMedium')

                # Localisation du service
                # localisation = coordonnee[0].text
                localisation = driver.find_element(By.CLASS_NAME,'rogA2c').text

                # Telephone du service
                # telephone = coordonnee[2].text
                
                time.sleep(2)
                if driver.find_elements(By.CLASS_NAME, 'yx21af.XDi3Bc'):

                    bouton_avis = driver.find_elements(By.CLASS_NAME,'Gpq6kf.fontTitleSmall')[1].text
                    if bouton_avis == 'Avis':
                        # Clique sur le bouton avis
                        driver.find_elements(By.CLASS_NAME,'hh2c6')[1].click()
                        time.sleep(2)

                        # Scroller mes commentaires jusqu'en bas
                        scrollable_commentaire = driver.find_elements(By.CLASS_NAME,'m6QErb.DxyBCb.kA9KIf.dS8AEf')[2]

                        freq_part = 0
                        while True:

                            driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_commentaire)
                            
                            # Cliquer sur le bouton lire plus des commentaires
                            button_as_plus = driver.find_elements(By.CLASS_NAME,'w8nwRe.kyuRq')
                            for i in button_as_plus:
                                # i.click()
                                driver.execute_script("arguments[0].click();", i)
                            
                            # S'arreter quand il y'a plus de bouton partager
                            freq_now = scrollable_commentaire.text.count('Partager\n')
                            if freq_now == freq_part:
                                break
                            freq_part = freq_now

                            time.sleep(2)

                        # commentaires service
                        comments_list = driver.find_elements(By.CLASS_NAME,'wiI7pd')
                        
                        if len(comments_list) > 0:
                            for i in comments_list:
                                comments_list1 = i.text.replace('\n', '').replace('"', '') 
                                comments_list_tout.append(comments_list1)
                                
                            time.sleep(1)

                            # Nom service    
                            service = driver.find_element(By.CLASS_NAME,'iD2gKb.W1neJ').text
                            for i in range(len(comments_list)):
                                services_list = service 
                                services_list_tout.append(services_list)
                            
                            time.sleep(1)
                            
                            # Date de publication
                            dates_list = driver.find_elements(By.CLASS_NAME,'rsqaWe')
                            for i, comment in enumerate(comments_list):
                                if comment and len(dates_list) > i:
                                    dates_list1 = dates_list[i].text.replace('il y a ', '') 
                                dates_list_tout.append(dates_list1)
                                
                            time.sleep(1)
                                
                            date_actuelle = date.today()
                            date_calcul = ''
                            date_calcul_tout = []
                            for this_date in dates_list_tout:
                                # ON VERIFIE LE FORMAT DU TEMPS (jour, mois, année)
                                # Création des regex :
                                #___Vérification pour les années
                                verif_year = re.findall(r"\b(\d+|un) (an|ans)\b", this_date)
                                #___Vérification pour les mois
                                verif_month = re.findall(r"\b(\d+|un) (mois)\b", this_date)
                                #___Vérification pour les semaines
                                verif_week = re.findall(r"\b(\d+|une) (semaine|semaines)\b", this_date)
                                #___Vérification pour les jours
                                verif_days = re.findall(r"\b(\d+|un) (jour|jours)\b", this_date)

                                if len(verif_year) != 0:
                                    try:
                                        if(int(verif_year[0][0])):
                                            # Soustraction de plusieurs années
                                            date_calcul = date_actuelle-timedelta(days=365*int(verif_year[0][0]))
                                    except ValueError:
                                        # Soustraction d'un seul
                                        date_calcul = date_actuelle-timedelta(days=365)
                                        
                                elif len(verif_month) != 0:
                                    try:
                                        if(int(verif_month[0][0])):
                                            # Soustraction de plusieurs mois
                                            date_calcul = date_actuelle-timedelta(days=30*int(verif_month[0][0]))
                                    except ValueError:
                                        # Soustraction d'un seul
                                        date_calcul = date_actuelle-timedelta(days=30)
                                        
                                elif len(verif_week) != 0:
                                    try:
                                        if(int(verif_week[0][0])):
                                            # Soustraction de plusieurs semaines
                                            date_calcul = date_actuelle-timedelta(days=7*int(verif_week[0][0]))
                                    except ValueError:
                                        # Soustraction d'un seul
                                        date_actuelle-timedelta(days=7)
                                        
                                elif len(verif_days) != 0:
                                    try:
                                        if(int(verif_days[0][0])):
                                            # Soustraction de plusieurs jours
                                            date_calcul = date_actuelle-timedelta(days=int(verif_days[0][0]))
                                    except ValueError:
                                        # Soustraction d'un seul
                                        date_calcul = date_actuelle-timedelta(days=1)
                                else:
                                    # print("aucune correspondance")
                                    continue
                                    
                                    
                                date_calcul_tout.append(date_calcul)
                            
                            time.sleep(1)

                            # Nom de l'auteur de la publication
                            nom_list = driver.find_elements(By.CLASS_NAME,'d4r55')
                            for i, comment in enumerate(comments_list):
                                if comment and len(nom_list) > i:
                                    nom_list1 = nom_list[i].text 
                                auteur_list_tout.append(nom_list1)
                                
                            time.sleep(1)

                            # localisation du service
                            for i in range(len(comments_list)):
                                localisation_list = localisation 
                                localisation_list_tout.append(localisation_list)
                                
                            time.sleep(1)

                            # Telephone du service
                            # for i in range(len(comments_list)):
                            #     telephone_list = telephone 
                            #     telephone_list_tout.append(telephone_list)
                            
                            time.sleep(1)
                            
                            # Nombre d'étoile pour chaque commentaire
                            contenu_div = driver.find_elements(By.CLASS_NAME, 'jftiEf.fontBodyMedium ')
                            for i, comment in enumerate(comments_list):
                                score = contenu_div[i].find_elements(By.CLASS_NAME, 'hCCjke.vzX5Ic')
                                etoile_list_tout.append(len(score))
                                    

                            time.sleep(1)
                    
                    else:
                        continue
                
        else:
            time.sleep(1)
            # copier le nom du service
            service = driver.find_element(By.CLASS_NAME,'DUwDvf.fontHeadlineLarge').text
            
            # Coordonnées du service
            # coordonnee = driver.find_elements(By.CLASS_NAME,'Io6YTe.fontBodyMedium')

            # Localisation du service
            # localisation = coordonnee[0].text
            localisation = driver.find_element(By.CLASS_NAME,'rogA2c').text
            

            # Telephone du service
            # telephone = coordonnee[2].text
            
            time.sleep(2)
            if driver.find_elements(By.CLASS_NAME, 'yx21af.XDi3Bc'):
                bouton_avis = driver.find_elements(By.CLASS_NAME,'Gpq6kf.fontTitleSmall')[1].text
                if bouton_avis == 'Avis':
                    # Clique sur le bouton avis
                    driver.find_elements(By.CLASS_NAME,'hh2c6')[1].click()
                    time.sleep(2)

                    # Scroller mes commentaires jusqu'en bas
                    scrollable_commentaire = driver.find_elements(By.CLASS_NAME,'m6QErb.DxyBCb.kA9KIf.dS8AEf')[0]

                    freq_part = 0
                    while True:

                        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_commentaire)
                        
                        # Cliquer sur le bouton lire plus des commentaires
                        button_as_plus = driver.find_elements(By.CLASS_NAME,'w8nwRe.kyuRq')
                        for i in button_as_plus:
                            # i.click()
                            driver.execute_script("arguments[0].click();", i)
                        
                        # S'arreter quand il y'a plus de bouton partager
                        freq_now = scrollable_commentaire.text.count('Partager\n')
                        if freq_now == freq_part:
                            break
                        freq_part = freq_now

                        time.sleep(3)

                    # commentaires service
                    comments_list = driver.find_elements(By.CLASS_NAME,'wiI7pd')
                    
                    if len(comments_list) > 0:
                        for i in comments_list:
                            comments_list1 = i.text.replace('\n', '').replace('"', '') 
                            comments_list_tout.append(comments_list1)
                            
                        time.sleep(1)

                        # Nom service    
                        for i in range(len(comments_list)):
                            services_list = service 
                            services_list_tout.append(services_list)
                        
                        time.sleep(1)
                        
                        # Date de publication
                        dates_list = driver.find_elements(By.CLASS_NAME,'rsqaWe')
                        for i, comment in enumerate(comments_list):
                            if comment and len(dates_list) > i:
                                dates_list1 = dates_list[i].text.replace('il y a ', '') 
                            dates_list_tout.append(dates_list1)
                            
                        time.sleep(1)
                            
                        date_actuelle = date.today()
                        date_calcul = ''
                        date_calcul_tout = []
                        for this_date in dates_list_tout:
                            # ON VERIFIE LE FORMAT DU TEMPS (jour, mois, année)
                            # Création des regex :
                            #___Vérification pour les années
                            verif_year = re.findall(r"\b(\d+|un) (an|ans)\b", this_date)
                            #___Vérification pour les mois
                            verif_month = re.findall(r"\b(\d+|un) (mois)\b", this_date)
                            #___Vérification pour les semaines
                            verif_week = re.findall(r"\b(\d+|une) (semaine|semaines)\b", this_date)
                            #___Vérification pour les jours
                            verif_days = re.findall(r"\b(\d+|un) (jour|jours)\b", this_date)

                            if len(verif_year) != 0:
                                try:
                                    if(int(verif_year[0][0])):
                                        # Soustraction de plusieurs années
                                        date_calcul = date_actuelle-timedelta(days=365*int(verif_year[0][0]))
                                except ValueError:
                                    # Soustraction d'un seul
                                    date_calcul = date_actuelle-timedelta(days=365)
                                    
                            elif len(verif_month) != 0:
                                try:
                                    if(int(verif_month[0][0])):
                                        # Soustraction de plusieurs mois
                                        date_calcul = date_actuelle-timedelta(days=30*int(verif_month[0][0]))
                                except ValueError:
                                    # Soustraction d'un seul
                                    date_calcul = date_actuelle-timedelta(days=30)
                                    
                            elif len(verif_week) != 0:
                                try:
                                    if(int(verif_week[0][0])):
                                        # Soustraction de plusieurs semaines
                                        date_calcul = date_actuelle-timedelta(days=7*int(verif_week[0][0]))
                                except ValueError:
                                    # Soustraction d'un seul
                                    date_actuelle-timedelta(days=7)
                                    
                            elif len(verif_days) != 0:
                                try:
                                    if(int(verif_days[0][0])):
                                        # Soustraction de plusieurs jours
                                        date_calcul = date_actuelle-timedelta(days=int(verif_days[0][0]))
                                except ValueError:
                                    # Soustraction d'un seul
                                    date_calcul = date_actuelle-timedelta(days=1)
                            else:
                                # print("aucune correspondance")
                                continue
                                
                                
                            date_calcul_tout.append(date_calcul)
                        
                        time.sleep(1)

                        # Nom de l'auteur de la publication
                        nom_list = driver.find_elements(By.CLASS_NAME,'d4r55')
                        for i, comment in enumerate(comments_list):
                            if comment and len(nom_list) > i:
                                nom_list1 = nom_list[i].text 
                            auteur_list_tout.append(nom_list1)
                            
                        time.sleep(1)

                        # localisation du service
                        for i in range(len(comments_list)):
                            localisation_list = localisation 
                            localisation_list_tout.append(localisation_list)
                            
                        time.sleep(1)

                        # Telephone du service
                        # for i in range(len(comments_list)):
                        #     telephone_list = telephone 
                        #     telephone_list_tout.append(telephone_list)
                        
                        time.sleep(1)
                        
                        # Nombre d'étoile pour chaque commentaire
                        contenu_div = driver.find_elements(By.CLASS_NAME, 'jftiEf.fontBodyMedium ')
                        for i, comment in enumerate(comments_list):
                            score = contenu_div[i].find_elements(By.CLASS_NAME, 'hCCjke.vzX5Ic')
                            etoile_list_tout.append(len(score))
                                

                        time.sleep(1)
                        
                else:
                    continue
            
        # for i in range(comments_list_tout):
        #     region_tout.append(region)
        
        # # Création de notre dictionnaire
        # dict_text = {
        #     'Services' : services_list_tout,
        #     'Localisation' : localisation_list_tout,
        #     'Télephone' : telephone_list_tout,
        #     'Auteur' : auteur_list_tout,
        #     'Commentaires': comments_list_tout,
        #     'Date' : date_calcul_tout,
        #     'Etoile' : etoile_list_tout
        # }

        # time.sleep(1)
        
        # # Recherche du nom de la region avec le lien du region reçu
        # for i in range(len(df_regions['Regions'])):
        #     if df_regions['Lien'][i] == region:
        #         nom_region = df_regions['Regions'][i]

        
        # # Créer un dataframe pour tous les services de chaque region
        # df = pd.DataFrame(dict_text)
        # # # df = df.drop_duplicates().reset_index(drop=True)
        # df.to_csv(f'../Data/{nom_region}_{service}.csv', index=False, sep=';')
    
    
    # Recherche du nom de la region avec le lien du region reçu
    for i in range(len(df_regions['Regions'])):
        if df_regions['Lien'][i] == region:
            nom_region = df_regions['Regions'][i]
            
    
    for i in range(comments_list_tout):
        region_tout.append(nom_region)       
        
            
    # Création de notre dictionnaire
    dict_text = {
        'Services' : services_list_tout,
        'Localisation' : localisation_list_tout,
        'Region' : region_tout,
        # 'Télephone' : telephone_list_tout,
        'Auteur' : auteur_list_tout,
        'Commentaires': comments_list_tout,
        'Date' : date_calcul_tout,
        'Etoile' : etoile_list_tout
    }

    time.sleep(1)
    


    
    # Créer un dataframe pour tous les services de chaque region
    df = pd.DataFrame(dict_text)
    # # df = df.drop_duplicates().reset_index(drop=True)
    df.to_csv(f'../Data/{nom_region}_tous_services.csv', index=False, sep=';')
        
    driver.quit()


KeyboardInterrupt: 